In [3]:
# imports and loading
from pymodulon.io import *
from pymodulon.plotting import *
from scipy.stats import fisher_exact, chi2_contingency
from statsmodels.stats.multitest import multipletests
import os
from os import path
import pandas as pd
import seaborn as sns
import ast 
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

base_dir = '/home/chris/zuniga/iModulonMiner'
external_data = path.join(base_dir,'data','external')
df_trn = pd.read_csv(path.join(external_data,'TRN.csv'))

data_dir = path.join(base_dir,'data','processed_data')
interim_dir = path.join(base_dir,'data','interim')
ica_data = load_json_model(path.join(interim_dir,'zmo_raw.json.gz'))
ica_data.gene_table.to_csv(path.join(data_dir, 'supp_gene_table.csv'))


/home/chris/miniforge3/envs/iM_miner/lib/python3.10/site-packages/pymodulon/util.py:31: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_json(table)
/home/chris/miniforge3/envs/iM_miner/lib/python3.10/site-packages/pymodulon/util.py:31: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_json(table)
/home/chris/miniforge3/envs/iM_miner/lib/python3.10/site-packages/pymodulon/util.py:31: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_json(table)
/home/chris/miniforge3/envs/iM_miner/lib/python3.10/site-packages/pymodulon/util.py:31: FutureWarning: Passing literal json to 'read_json' 

In [21]:
len(blast_df)

1916

array(['ZCP4_1144', 'ZCP4_0987', 'ZCP4_0983', 'ZCP4_0988', 'ZCP4_0989',
       'ZCP4_0984', 'ZCP4_0986', 'ZCP4_0049', 'ZCP4_0660', 'ZCP4_1224',
       'ZCP4_1669', 'ZCP4_1625', 'ZCP4_1839', 'ZCP4_1918'], dtype=object)

In [45]:
# look at blast results
blast_df = pd.read_csv(base_dir+'/Zymomonas_mobilis/blast_results/results.txt', names = ['ZM4_gene', 'CP4_gene', 'match', 'alightment_length', 'mismatches', 'open_gaps', 'query_start', 'query_end', 'subject_start', 'subject_end', 'evalue', 'bitscore'], delimiter = '\t')

bad_match_genes = blast_df[blast_df['match'] < 90].CP4_gene.unique()
ica_data.gene_table.loc[bad_match_genes]


,gene_name,accession,start,end,strand,gene_product,COG,regulator,uniprot,operon,...,LDI_early_glucose_fold_change,LDI_late_glucose_fold_change,LDI_early_xylose_fold_change,LDI_late_xylose_fold_change,no_LDI_early_glucose_fold_change,no_LDI_late_glucose_fold_change,no_LDI_early_xylose_fold_change,no_LDI_late_xylose_fold_change,chromosome_id,iMs
ZCP4_1144,ZCP4_1144,CP006818.1,1289021,1289725,+,hypothetical protein,No COG annotation,"ZCP4_0128,ZCP4_1718",NaN,Op1243,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CP006818.1,NaN
ZCP4_0515,ZCP4_0515,CP006818.1,585554,588076,-,outer membrane receptor protein,Inorganic ion transport and metabolism,"ZCP4_0018,ZCP4_0128,ZCP4_0523,ZCP4_0523,ZCP4_0...",NaN,Op635,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CP006818.1,NaN
ZCP4_0999,ZCP4_0999,CP006818.1,1118734,1127376,-,Putative flagellar system-associated repeat pr...,Cell wall/membrane/envelope biogenesis,"ZCP4_0018,ZCP4_0128,ZCP4_0375,ZCP4_0523,ZCP4_0...",NaN,Op1102,...,1.0,0.879184,0.727236,0.537499,1.0,0.728417,0.617872,0.502399,CP006818.1,['toxin_secretion']
ZCP4_0987,ZCP4_0987,CP006818.1,1102363,1103790,+,Aspartyl protease,Function unknown,NaN,Q5NQW3,Op1090,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CP006818.1,['plasmid']
ZCP4_0988,ZCP4_0988,CP006818.1,1103845,1105272,+,hypothetical protein,Function unknown,ZCP4_1387,Q5NQW5,Op1091,...,1.0,0.958379,0.690203,0.288789,1.0,1.359752,1.038499,0.365161,CP006818.1,NaN
ZCP4_0985,ZCP4_0985,CP006818.1,1099405,1100829,+,hypothetical protein,Function unknown,NaN,Q5NQW5,Op1088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CP006818.1,['plasmid']
ZCP4_0984,ZCP4_0984,CP006818.1,1097926,1099350,+,hypothetical protein,Function unknown,NaN,Q5NQW1,Op1087,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CP006818.1,NaN
ZCP4_0989,ZCP4_0989,CP006818.1,1105327,1106760,+,hypothetical protein,Function unknown,NaN,Q5NQW6,Op1092,...,1.0,1.206386,0.980573,0.504769,1.0,1.077420,0.856074,0.416152,CP006818.1,NaN
ZCP4_0990,ZCP4_0990,CP006818.1,1106815,1108218,+,Aspartyl protease,Function unknown,ZCP4_1387,Q5NQW3,Op1093,...,1.0,1.027323,0.908534,0.682450,1.0,1.075533,0.966533,0.590291,CP006818.1,NaN
ZCP4_0983,ZCP4_0983,CP006818.1,1096447,1097871,+,hypothetical protein,Function unknown,"ZCP4_0375,ZCP4_0900,ZCP4_1718",Q5NQW0,Op1086,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CP006818.1,NaN


In [56]:

# let's create groups of regulators

# first, by iModulon
iM_to_genes = {}
for iM in ica_data.imodulon_table.index:
    iM_to_genes.update({iM : list(ica_data.view_imodulon(iM).sort_index().index)})
pd.Series(iM_to_genes).to_csv(path.join(data_dir, 'supp_iM_to_genes.csv'))

# second by DAP-seq
DAP_to_genes = {}
for reg, df in df_trn.groupby('regulator'):
    DAP_to_genes.update({reg : set(df['gene_id'])})
for k, v in DAP_to_genes.items():
    new = list(v)
    new.sort()
    DAP_to_genes.update({k : new})
pd.Series(DAP_to_genes).to_csv(path.join(data_dir, 'supp_DAP_to_genes.csv'))

# third by seq-based
tot_regs = set(ica_data.gene_table.FIMO_regulators)
all_regs = set()
for regs in tot_regs:
    for reg in ast.literal_eval(regs):
        all_regs.add(reg)
FIMO_to_genes = {reg : set() for reg in all_regs}
for index, row in ica_data.gene_table.iterrows():
    for reg in ast.literal_eval(row['FIMO_regulators']):
        FIMO_to_genes[reg].add(index)
for k, v in FIMO_to_genes.items():
    new = list(v)
    new.sort()
    FIMO_to_genes.update({k : new})
pd.Series(FIMO_to_genes).to_csv(path.join(data_dir, 'supp_FIMO_to_genes.csv'))